<a href="https://colab.research.google.com/github/naratiz/Precoce-MS/blob/main/4_Modeling_Classifica%C3%A7%C3%A3o/Otimiza%C3%A7%C3%A3o_Classifica%C3%A7%C3%A3o_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install optuna

In [34]:
import optuna
import pandas as pd
import numpy as np
import logging
import sys
import time

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils import class_weight
from sklearn.model_selection import cross_val_score,train_test_split, StratifiedKFold

from catboost import CatBoostClassifier

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import concurrent.futures
import itertools


from sklearn.model_selection import cross_val_score,train_test_split, StratifiedKFold, learning_curve

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from imblearn.combine import SMOTEENN 

In [1]:
import os 
n_cpu = os.cpu_count()
print("Number of CPUs in the system:", n_cpu)

Number of CPUs in the system: 4


# Base de Dados

In [2]:
path = '../../dados/df_train_class.csv'
df = pd.read_csv(path, sep=';')

In [3]:
print('A base de dados apresenta {} registros e {} características.'.format(df.shape[0], df.shape[1]))

A base de dados apresenta 672657 registros e 48 características.


In [24]:
df=df.sample(1000)

### Separando as características preditoras do atributo alvo

In [25]:
SEED = 42

In [26]:
y = df['categoria']
# Retirar a variável alvo e as variáveis obtidas após o abate 
X = df.drop(['categoria'],axis=1)

In [27]:
# Para remover UserWarning do XGBClassifier: Use o parêmetro use_label_encoder=False 
# e codifique as classes no inervalo[0 ... n]
y=y.replace(6, 0)

### StandardScaler

In [28]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## PCA - Não compilado

In [15]:
pca = PCA(n_components=30)
pca.fit(X_scaled)
X_scaled_pca = pca.transform(X_scaled)

## Baseline

In [45]:
class_weight = class_weight.compute_class_weight(class_weight = "balanced",
                                                 classes = np.unique(y),
                                                 y = y)

In [30]:
skfolds = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

In [33]:
cbc_baseline = CatBoostClassifier(loss_function='MultiClass', class_weights= class_weight,
                                  iterations=10, depth=10, random_state=SEED, verbose=False)
score = cross_val_score(cbc_baseline, X_scaled, y, cv = skfolds)

print('Acurácia média da validação cruzada : %0.3f"' % score.mean())

Acurácia média da validação cruzada : 0.278"


## Otimização dos hiperparâmetros

In [35]:
skfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

In [43]:
# Manipulador de fluxo para mostrar as mensagens
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "cbc-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
# storage_name = "sqlite:///content/drive/MyDrive/Dissertação/dados/{}.db".format(study_name)

In [37]:
# função de otimização baseada em uma função objetivo
def tune (objective):
    study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize")
    study.optimize(objective, timeout=86400, n_trials=300)

    params = study.best_params
    print(f"Trial: {study.best_trial.number}")
    params = study.best_params
    print("\n")
    print(f"Melhor seleção de parâmetros : \n{params}")
    print("\n")
    print(f"Acurácia : {study.best_value:.3f}")
    
    return study, params

# CatBoostClassifier

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=SEED)

In [ ]:
random_strength?
iterations
model_size_reg=None,
max_depth=None,
n_estimators=None,

In [47]:
def cbc_objective(trial: optuna.Trial): 
    cbc_params = {
        'loss_function':'MultiClass',
        'iterations': trial.suggest_int('iterations', 1, 12),
        'depth': trial.suggest_int('depth', 1, 12),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-5, 1e0),
        'l2_leaf_reg': trial.suggest_loguniform("l2_leaf_reg", 1e-2, 1e0),
        'random_state': SEED
    }
    model = CatBoostClassifier(**cbc_params, class_weights= class_weight, verbose=False) 
    scores = cross_val_score(model, X_train, y_train, n_jobs=-1, cv=skfolds)
    return scores.mean()

In [46]:
t0 = time.time()
study, best_params = tune(cbc_objective)  
print("Pronto em:  %0.3f" %(time.time()-t0))

[I 2022-07-13 23:14:52,277] Using an existing study with name 'cbc-study' instead of creating a new one.


Using an existing study with name 'cbc-study' instead of creating a new one.
Using an existing study with name 'cbc-study' instead of creating a new one.
Using an existing study with name 'cbc-study' instead of creating a new one.


[W 2022-07-13 23:14:52,406] Trial 1 failed because of the following error: TypeError("__init__() got an unexpected keyword argument 'class_weight'")
Traceback (most recent call last):
  File "/home/inara/anaconda3/lib/python3.7/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-41-4fcec4b2854a>", line 11, in cbc_objective
    model = CatBoostClassifier(**cbc_params)
TypeError: __init__() got an unexpected keyword argument 'class_weight'


Trial 1 failed because of the following error: TypeError("__init__() got an unexpected keyword argument 'class_weight'")
Traceback (most recent call last):
  File "/home/inara/anaconda3/lib/python3.7/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-41-4fcec4b2854a>", line 11, in cbc_objective
    model = CatBoostClassifier(**cbc_params)
TypeError: __init__() got an unexpected keyword argument 'class_weight'
Trial 1 failed because of the following error: TypeError("__init__() got an unexpected keyword argument 'class_weight'")
Traceback (most recent call last):
  File "/home/inara/anaconda3/lib/python3.7/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-41-4fcec4b2854a>", line 11, in cbc_objective
    model = CatBoostClassifier(**cbc_params)
TypeError: __init__() got an unexpected keyword argument 'class_weight'
Trial 1 failed because of the 

TypeError: __init__() got an unexpected keyword argument 'class_weight'

In [ ]:
# Determina quais parâmetros tiveram o impacto mais significativo no desempenho geral do modelo.
optuna.visualization.plot_param_importances(study)